# Telegram - Create Crypto sentiment bot

## Input

### Install telegram

In [9]:
pip install telegram

Note: you may need to restart the kernel to use updated packages.


### Import library

In [11]:
import logging
from telegram import *
import numpy as np
import naas_drivers

### Variables

[Create a Telegram bot and get your API KEY](https://core.telegram.org/bots#6-botfather)

In [13]:
TELEGRAM_API_KEY = "***********"

[Get Newsapi API KEY](https://newsapi.org/)

In [ ]:
NEWSAPI_API_KEY = "************"

### Connect to Newsapi Naas drivers

In [15]:
naas_drivers.newsapi.connect(NEWSAPI_API_KEY)

### Set up the logging

In [12]:
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
logging.info('Starting Bot...')

2021-07-20 16:27:08,708 - root - INFO - Starting Bot...


## Model

### Function to ...

In [17]:
#user_response
def get_response(user_message):
    
    usr_msg_text = user_message.lower()
    
    if usr_msg_text in ["bitcoin","etherium", "ripple","dogecoin","cardano", "polygon"]:
        data = naas_drivers.newsapi.get(usr_msg_text)
        sentiment =  naas_drivers.sentiment_analysis.calculate(data, column_name="title")
    else:
        sentiment = 'There is no data for this ticker\n use \help command '
    
    return sentiment

### Commands

In [18]:
def start_command(update, context):
    update.message.reply_text('Hello there! I\'m a crypto sentiment bot. send me the coin name, I\'ll send you sentiment!')


def help_command(update, context):
    update.message.reply_text('check the coin names from the list below!\n bitcoin\n etherium \n ripple\n dogecoin\n cardano\n polygon\n')

### Message handler

In [ ]:
def handle_message(update, context):
    text = str(update.message.text).lower()
    logging.info(f'User ({update.message.chat.id}) says: {text}')

    # Bot response
    sentiment_naas = get_response(text)
    update.message.reply_text(sentiment_naas)

### Manage error logs

In [ ]:
def error(update, context):
    # Logs errors
    logging.error(f'Update {update} caused error {context.error}')

## Output

### Run the program

In [ ]:
if __name__ == '__main__':
    # create an object "bot"
    updater = Updater(API_KEY, use_context=True)
    dp = updater.dispatcher

    #  bot's command handlers
    dp.add_handler(CommandHandler('start', start_command))
    dp.add_handler(CommandHandler('help', help_command))

    # bot's text handlers 
    dp.add_handler(MessageHandler(Filters.text, handle_message))

    # bot's error handler
    dp.add_error_handler(error)

    # Run the bot
    updater.start_polling(1.0)
    updater.idle()

2021-07-08 10:32:05,786 - apscheduler.scheduler - INFO - Scheduler started
2021-07-08 10:32:20,772 - root - INFO - User (1840263262) says: bitcoin
2021-07-08 10:32:21,038 - root - ERROR - Update {'update_id': 415492296, 'message': {'photo': [], 'entities': [], 'supergroup_chat_created': False, 'chat': {'id': 1840263262, 'first_name': 'yash', 'type': 'private', 'last_name': 'kumar'}, 'delete_chat_photo': False, 'new_chat_photo': [], 'text': 'bitcoin', 'group_chat_created': False, 'message_id': 48, 'new_chat_members': [], 'date': 1625733140, 'channel_chat_created': False, 'caption_entities': [], 'from': {'last_name': 'kumar', 'language_code': 'en', 'id': 1840263262, 'first_name': 'yash', 'is_bot': False}}} caused error {'status': 'error', 'code': 'parametersMissing', 'message': 'Required parameters are missing, the scope of your search is too broad. Please set any of the following required parameters and try again: q, qInTitle, sources, domains.'}
